## This notebook is used to generate the finalized version of the classifier, to simply feature transformation into the final form, and to test that the results are the same

Most of the code comes from operational_classifier.

In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")

#Loading raw data
# df = pickle.load(open("../Data/multiclass_tweets_indexed.p",'rb'))
# df = pickle.load(open("../Data/labeled_data.p",'rb'))
df = pd.read_pickle("../Data/labeled_data.p")
# tweets = df.text
tweets = df.tweet

In [126]:
# df.shape
df.head(3)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...


In [141]:
tweets = df.tweet
tweets

0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5        !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6        !!!!!!"@__BrighterDays: I can not just sit up ...
7        !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8        " &amp; you might not get ya bitch back &amp; ...
9        " @rhythmixx_ :hobbies include: fighting Maria...
10       " Keeks is a bitch she curves everyone " lol I...
11                      " Murda Gang bitch its Gang Land "
12       " So hoes that smoke are losers ? " yea ... go...
13           " bad bitches is the only thing that i like "
14                                 " bitch get up off me "
15                         " bitch nigga miss me with it "
16                                  " bitch plz whatever

## Feature generation

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [129]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
# idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

D:\programs\Continuum\miniconda3\envs\py35\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


In [130]:
# idf_dict = {}
# for i in vocab.values():
#     print(str(i) + '\t\t' + str(idf_vals[i]))
#     idf_dict[i] = idf_vals[i]

In [131]:
# idf_dict

In [132]:
# vocab

# for i, v in enumerate(vectorizer.get_feature_names()):
#     print(str(i) + '\t\t' + v)

In [133]:
# for i in vocab.values():
#     print(i)

In [134]:
# idf_dict

In [135]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
#     print(tags)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)
        #print(tokens[i],tag_list[i])

D:\programs\Continuum\miniconda3\envs\py35\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [136]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )

In [138]:
pd.Series(tweet_tags)

0        RB NN IN DT NN PRP VBP JJ NN IN VBG RP PRP$ NN...
1           JJ NN NN NNS VBP JJ JJ IN NN NN NN IN DT NN NN
2        JJ NN NN NN PRP RB VBD DT NN CC PRP VBP TO VB ...
3                                  NN VBZ PRP VBP IN DT NN
4        NN VBZ DT NN PRP VBP IN PRP MD VB JJ CC PRP MD...
5        IN DT NN RB VBZ NN PRP RB JJ CC RB IN NN CC RB...
6        NN NN MD RB RB VB RP CC VB IN DT NN NN NN VBD ...
7        NN NN NN VBP VBN IN PRP JJ NNS VBG IN PRP VBP NNS
8                      NN PRP MD RB VB JJ VB RB JJ NNS WDT
9                                       NNS VBP VBG NNS NN
10       NN VBZ DT NN PRP VBZ NN NN NN VBD IN DT NN IN ...
11                                     NN NN VB PRP$ NN NN
12                   RB VBZ IN NN VBP NNS . NN : VBP IN NN
13                            JJ NNS VBZ DT JJ NN IN NN IN
14                                         NN VB RP IN PRP
15                                     NN NN VB PRP IN PRP
16                                               NN NN W

In [142]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [146]:
pos_vocab

{'CD NN RB': 205,
 'TO DT NN': 2460,
 'VBD JJ VB': 2854,
 'NNS VBZ JJ': 1746,
 'CD NNS RB': 216,
 'RB NN JJ': 2146,
 'JJ IN VBG': 773,
 'CC NN NNS': 51,
 'RB VBD JJ': 2243,
 'JJ CC DT': 732,
 'WP VBZ PRP': 3957,
 'EX MD': 446,
 'IN DT JJS': 502,
 'VB NN IN': 2619,
 'MD VB JJR': 1097,
 'CD NN MD': 201,
 'DT PRP JJ': 339,
 'NN RP PRP': 1339,
 'EX VBZ NN': 460,
 'EX VBZ DT': 459,
 'CD CC': 182,
 'WP VBD DT': 3926,
 'NNS VBP VBZ': 1740,
 'VBZ WP': 3796,
 'RP DT NNS': 2375,
 'VB DT RBR': 2560,
 'WRB MD': 3992,
 'VBZ WP NN': 3799,
 'CC PRP NN': 81,
 'CC VBD PRP': 135,
 'RB VBP RP': 2292,
 'DT VBP TO': 415,
 'CC VB JJ': 113,
 'VBZ RB IN': 3708,
 'VBD EX': 2827,
 'NN WP DT': 1485,
 'RBS JJ NN': 2360,
 'VBD IN CC': 2829,
 'VBZ IN IN': 3626,
 'WP VBD IN': 3927,
 'IN NN VBG': 588,
 'RB VBN VB': 2277,
 'VBP IN WP': 3367,
 'VBP IN EX': 3351,
 'RB VBP CD': 2283,
 'JJ WRB NNS': 982,
 'VBD NN WDT': 2881,
 'DT VBN': 401,
 'CC NN RB': 53,
 'NNS IN': 1554,
 'RBR IN PRP': 2336,
 'NN JJ DT': 1190,
 'PRP CC

In [147]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *

sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [148]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", "vader compound", \
                        "num_hashtags", "num_mentions", "num_urls", "is_retweet"]

In [149]:
feats = get_feature_array(tweets)

In [151]:
tfidf.shape

(24783, 7086)

In [156]:
tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [152]:
pos.shape

(24783, 4063)

In [155]:
pos

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [153]:
feats.shape

(24783, 17)

In [154]:
feats

array([[  8.3 ,  79.94,  30.  , ...,   1.  ,   0.  ,   0.  ],
       [  4.7 ,  90.13,  19.  , ...,   1.  ,   0.  ,   0.  ],
       [  5.8 ,  89.25,  23.  , ...,   2.  ,   0.  ,   1.  ],
       ...,
       [  3.1 ,  96.03,  15.  , ...,   0.  ,   0.  ,   0.  ],
       [  0.6 , 103.05,   8.  , ...,   0.  ,   0.  ,   0.  ],
       [  9.8 ,  55.22,  27.  , ...,   0.  ,   1.  ,   0.  ]])

In [19]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)

In [157]:
M.shape

(24783, 11166)

In [162]:
#Finally get a list of variable names
variables = ['']*len(vocab)
# for k,v in vocab.iteritems():
#     variables[v] = k
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
# for k,v in pos_vocab.iteritems():
#     pos_variables[v] = k
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names
len(feature_names)

11166

# Running the model

This model was found using a GridSearch with 5-fold cross validation. Details are in the notebook operational_classifier.

In [24]:
X = pd.DataFrame(M)
y = df['class'].astype(int)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [163]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.01))
X_ = select.fit_transform(X,y)

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [167]:
X_.shape

(24783, 175)

In [168]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_, y)

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [169]:
y_preds = model.predict(X_)

In [170]:
report = classification_report( y, y_preds )

In [171]:
print(report)

              precision    recall  f1-score   support

           0       0.47      0.55      0.51      1430
           1       0.97      0.92      0.94     19190
           2       0.82      0.96      0.89      4163

   micro avg       0.90      0.90      0.90     24783
   macro avg       0.75      0.81      0.78     24783
weighted avg       0.91      0.90      0.91     24783



# Using information from the model to obtain the matrix X_ generically

This is the most difficult task: We have to take the inputs tweets and transform them into a format that can be used in the model without going through all the same pre-processing steps as above. This can be done as follows.

## Obtaining information about the model

In [175]:
final_features = select.get_support(indices=True) #get indices of features
# final_feature_list = [unicode(feature_names[i]) for i in final_features] #Get list of names corresponding to indices
final_feature_list = [str(feature_names[i]) for i in final_features] #Get list of names corresponding to indices

In [176]:
# print final_feature_list
print(final_feature_list)

['america', 'american', 'anoth', 'ass', 'ass cracker', 'ass hoe', 'ass nigga', 'bad', 'beaner', 'big', 'bird', 'bitch', 'bitch nigga', 'black', 'border', 'born', 'bout', 'browni', 'campu', 'charli', 'chink', 'color', 'color folk', 'coon', 'countri', 'cracker', 'crazi', 'crippl', 'cunt', 'da', 'damn', 'darki', 'dick', 'die', 'doe', 'dyke', 'fag', 'faggot', 'fat', 'femal', 'feminist', 'filth', 'first', 'folk', 'fucc nicca', 'fuck', 'fuckin', 'game', 'gay', 'get', 'ghetto', 'girl', 'gon', 'good', 'gook', 'got nigga', 'gt gt', 'hate', 'hate hoe', 'hi', 'hire', 'ho', 'hoe', 'hood', 'hope', 'human', 'israel', 'jap', 'jew', 'jihadi', 'kill', 'lame', 'latina', 'let', 'like', 'lol', 'look like', 'love', 'may', 'mexican', 'mock', 'money', 'monkey', 'much', 'muslim', 'muzzi', 'negro', 'nicca', 'nig', 'nigga', 'nigga bitch', 'niggah', 'niggaz', 'nigger', 'nigguh', 'niglet', 'oreo', 'peopl', 'play', 'pussi', 'queer', 'race', 'racist', 'real', 'real nigga', 'redneck', 'retard', 'sex', 'shit', 'shoot

In [181]:
#Getting names for each class of features
ngram_features = final_feature_list[:final_feature_list.index('zimmerman')+1]
pos_features = final_feature_list[final_feature_list.index('zimmerman')+1:final_feature_list.index('VBD')+1]
oth_features = final_feature_list[final_feature_list.index('VBD')+1:]

In [183]:
oth_features

['FKRA',
 'FRE',
 'num_syllables',
 'num_chars',
 'num_chars_total',
 'num_terms',
 'num_words',
 'num_unique_words',
 'vader compound',
 'num_hashtags',
 'num_mentions']

## Generating ngram features

In [186]:
new_vocab = {v:i for i, v in enumerate(ngram_features)}
new_vocab_to_index = {}
for k in ngram_features:
    new_vocab_to_index[k] = vocab[k]

In [187]:
new_vocab_to_index

{'america': 126,
 'american': 127,
 'anoth': 180,
 'ass': 250,
 'ass cracker': 260,
 'ass hoe': 269,
 'ass nigga': 277,
 'bad': 356,
 'beaner': 416,
 'big': 493,
 'bird': 512,
 'bitch': 535,
 'bitch nigga': 753,
 'black': 893,
 'border': 958,
 'born': 960,
 'bout': 973,
 'browni': 1047,
 'campu': 1132,
 'charli': 1209,
 'chink': 1253,
 'color': 1332,
 'color folk': 1335,
 'coon': 1408,
 'countri': 1433,
 'cracker': 1444,
 'crazi': 1450,
 'crippl': 1470,
 'cunt': 1499,
 'da': 1525,
 'damn': 1544,
 'darki': 1561,
 'dick': 1665,
 'die': 1674,
 'doe': 1721,
 'dyke': 1820,
 'fag': 2000,
 'faggot': 2002,
 'fat': 2040,
 'femal': 2081,
 'feminist': 2084,
 'filth': 2099,
 'first': 2117,
 'folk': 2168,
 'fucc nicca': 2245,
 'fuck': 2246,
 'fuckin': 2332,
 'game': 2366,
 'gay': 2379,
 'get': 2390,
 'ghetto': 2470,
 'girl': 2485,
 'gon': 2580,
 'good': 2599,
 'gook': 2620,
 'got nigga': 2658,
 'gt gt': 2728,
 'hate': 2801,
 'hate hoe': 2809,
 'hi': 2859,
 'hire': 2898,
 'ho': 2911,
 'hoe': 2916,
 

In [188]:
#Get indices of text features
ngram_indices = final_features[:len(ngram_features)]

In [190]:
ngram_indices.shape

(156,)

In [38]:
#TODO: Pickle new vectorizer

In [191]:
new_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_vocab
    )

In [193]:
from sklearn.externals import joblib
joblib.dump(new_vectorizer, 'final_tfidf.pkl') 

['final_tfidf.pkl']

In [194]:
tfidf_ = new_vectorizer.fit_transform(tweets).toarray()

D:\programs\Continuum\miniconda3\envs\py35\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


In [195]:
#Verifying that results are the same

In [196]:
tfidf_[1,:]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [197]:
tfidf_[1,:].sum()

2.0

In [198]:
X_[1,:tfidf_.shape[1]]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 4.80981477, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 2.81738461, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [199]:
X_[1,:tfidf_.shape[1]].sum()

7.627199378751962

Results are the same if use IDF but the problem is that IDF will be different if we use different data. Instead we have to use the original IDF scores and multiply them by the new matrix.

In [200]:
idf_vals_ = idf_vals[ngram_indices]

In [201]:
idf_vals_.shape

(156,)

In [202]:
#TODO: Pickle idf_vals

joblib.dump(idf_vals_, 'final_idf.pkl') 

['final_idf.pkl']

In [203]:
(tfidf_[1,:]*idf_vals_) == X_[1,:153] #Got same value as final process array!

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [204]:
tfidf_*idf_vals_ == X_[:,:153]

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [205]:
tfidffinal = tfidf_*idf_vals_

## Generating POS features
This is simpler as we do not need to worry about IDF but it will be slower as we have to compute the POS tags for the new data. Here we can simply use the old POS tags.

In [53]:
new_pos = {v:i for i, v in enumerate(pos_features)}

In [54]:
#TODO: Pickle pos vectorizer
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
new_pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_pos
    )

In [55]:
joblib.dump(new_pos_vectorizer, 'final_pos.pkl') 

['final_pos.pkl']

In [56]:
pos_ = new_pos_vectorizer.fit_transform(tweet_tags).toarray()

In [57]:
pos_[1,:]

array([0., 1., 1., 1., 1., 0., 0., 0., 0.])

In [58]:
X_[1,153:159]

array([0., 0., 0., 1., 1., 1.])

In [59]:
pos_[:,:] == X_[:,153:159]

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [60]:
pos_[:,:].sum()

95689.0

In [61]:
X_[:,153:159].sum()

33411.363676851324

## Finally, we can look at the other features

In [62]:
# print other_features_names
print(other_features_names)

['FKRA', 'FRE', 'num_syllables', 'avg_syl_per_word', 'num_chars', 'num_chars_total', 'num_terms', 'num_words', 'num_unique_words', 'vader neg', 'vader pos', 'vader neu', 'vader compound', 'num_hashtags', 'num_mentions', 'num_urls', 'is_retweet']


In [63]:
# print oth_features
print(oth_features)

['FKRA', 'FRE', 'num_syllables', 'num_chars', 'num_chars_total', 'num_terms', 'num_words', 'num_unique_words', 'vader compound', 'num_hashtags', 'num_mentions']


The functions can be modified to only calculate and return necessary fields.

In [64]:
def other_features_(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    features = [FKRA, FRE, syllables, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['compound'],
                twitter_objs[2], twitter_objs[1],]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array_(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features_(t))
    return np.array(feats)

In [65]:
feats_ = get_feature_array_(tweets)

In [66]:
feats_[0,:]

array([  8.3   ,  79.94  ,  30.    , 127.    , 140.    ,  25.    ,
        25.    ,  23.    ,   0.4563,   0.    ,   1.    ])

In [67]:
X_[0,159:]

array([  0.    ,   1.    ,   2.    ,   1.    ,   0.    ,   8.3   ,
        79.94  ,  30.    , 127.    , 140.    ,  25.    ,  25.    ,
        23.    ,   0.4563,   0.    ,   1.    ])

In [68]:
feats_[:,:] == X_[:,159:]

D:\programs\Continuum\miniconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

## Now that we have put it all together using a simplified process we can assess if these new data return the same answers.

In [69]:
M_ = np.concatenate([tfidffinal, pos_, feats_],axis=1)

In [70]:
M_.shape

(24783, 175)

In [71]:
X__ = pd.DataFrame(M_)

In [72]:
y_preds_ = model.predict(X__)

In [73]:
report = classification_report( y, y_preds_ )

In [74]:
print(report)

              precision    recall  f1-score   support

           0       0.47      0.56      0.51      1430
           1       0.97      0.92      0.94     19190
           2       0.82      0.96      0.89      4163

   micro avg       0.90      0.90      0.90     24783
   macro avg       0.75      0.81      0.78     24783
weighted avg       0.91      0.90      0.91     24783



OK. So now that we have verified that the results are the same with X_ and X__ we can implement a script that can transform new data in this manner.